In [1]:
from artifacts_ipv6_sra_scanning.config import *
import pickle
import os
import ipaddress
import glob
from tqdm.auto import tqdm
import country_converter as coco
from parallel_pandas import ParallelPandas
ParallelPandas.initialize(n_cpu=24, split_factor=1, disable_pr_bar=False)

2025-10-05 19:04:48.469 | INFO     | artifacts_ipv6_sra_scanning.config:<module>:22 - PROJ_ROOT path is: /home/service/artifacts-ipv6-sra-scanning


In [2]:
df = pl.read_parquet(f'{PROCESSED_DATA_DIR}/sra_tum_scans_stability.parquet')

In [3]:
df.head()

initial-ip,saddr,classification,saddr_s1,classification_s1,saddr_s2,classification_s2,saddr_s3,classification_s3,saddr_s4,classification_s4,saddr_s5,classification_s5
str,str,str,str,str,str,str,str,str,str,str,str,str
"""240e:358:e00:6b54::""","""240e:358:e00:6b54:7869:e171:7f…","""echoreply""","""240e:358:e00:6b54:7869:e171:7f…","""echoreply""","""240e:358:e00:6b54:7869:e171:7f…","""echoreply""",null,null,null,null,null,null
"""2409:40e4:8:7db2::""","""2409:40e4:8:7db2:5e2f:65c5:7b5…","""unreach_noroute""","""2409:40e4:8:7db2:5e2f:65c5:7b5…","""unreach_noroute""","""2409:40e4:8:7db2:5e2f:65c5:7b5…","""unreach_noroute""",null,null,null,null,null,null
"""2001:1670:10:6362::""","""2001:1670:10:6362:1008:d0df:88…","""echoreply""","""2001:1670:10:6362:1008:d0df:88…","""echoreply""","""2001:1670:10:6362:1008:d0df:88…","""echoreply""","""2001:1670:14:0:40a4:5e47:67f0:…","""unreach_addr""","""2001:1670:10:6362:4dbb:9498:1d…","""echoreply""","""2001:1670:10:6362:4dbb:9498:1d…","""echoreply"""
"""2003:e3:3719:2e00::""","""2003:e3:37ff:19a5:e228:6dff:fe…","""unreach_admin""","""2003:e3:37ff:19a5:e228:6dff:fe…","""unreach_admin""","""2003:e3:37ff:19a5:e228:6dff:fe…","""unreach_admin""","""2003:e3:37ff:19a5:e228:6dff:fe…","""unreach_admin""","""2003:e3:37ff:19a5:e228:6dff:fe…","""unreach_admin""","""2003:e3:37ff:19a5:e228:6dff:fe…","""unreach_admin"""
"""2400:2412:4761:b600::""","""2400:2412:4761:b601:3ae0:8eff:…","""echoreply""","""2400:2412:4761:b601:3ae0:8eff:…","""echoreply""","""2400:2412:4761:b601:3ae0:8eff:…","""echoreply""","""2400:2412:4761:b601:3ae0:8eff:…","""echoreply""","""2400:2412:4761:b601:3ae0:8eff:…","""echoreply""","""2400:2412:4761:b601:3ae0:8eff:…","""echoreply"""


In [6]:
df.select('initial-ip').write_csv(f'{INTERIM_DATA_DIR}/init-ips-sra-tum-scan.csv')

In [7]:
def load_and_join(files):
    df = pl.DataFrame()
    for i in tqdm(range(len(files))):
        if df.is_empty():
            df = pl.read_csv(files[i],columns=['initial-ip','saddr','classification']).unique(subset=['initial-ip'])
        else:
            tmp = pl.read_csv(files[i],columns=['initial-ip','saddr','classification']).unique(subset=['initial-ip'])
            df = df.join(tmp,on='initial-ip',how='left', suffix=f'_s{i}')
    return df

In [8]:
file1 = '/home/service/ipv6-measurements-mkoch/backup_2024/sra-analysis/data/bgp48_sra_checks/zmap_icmp_bgp_48_sra_1728075185.log.zst'

In [9]:
df = load_and_join([file1])

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
df.head()

initial-ip,saddr,classification
str,str,str
"""2a0c:885:7d93::""","""2001:67c:2070:b901::b""","""timxceed"""
"""2804:7b68:7d35::""","""2804:c84:10:101::2e""","""unreach_noroute"""
"""2a14:0:469d::""","""2001:19f0:6c00:18b2:5400:4ff:f…","""unreach_noroute"""
"""2804:6e4:7402::""","""2804:6e4:1::5""","""timxceed"""
"""2604:14c0:7603::""","""2607:ffc8:0:6::10""","""unreach_rejectroute"""


In [11]:
len(df)

337274845

In [12]:
df.select('initial-ip').write_csv(f'{INTERIM_DATA_DIR}/init-ips-sra-bgp48-scan.csv')